<a href="https://colab.research.google.com/github/xeniia-nikole/Python_detecting_objects/blob/master/searching_in_video_by_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install opencv-python
import cv2
import os
import numpy as np
from matplotlib import pyplot
from IPython.display import Image 
from IPython import display

path = '/content/'
video = path + 'video_budgies.mp4'

In [ ]:
# Load names of classes and get random colors
labels = open(path + 'coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [ ]:
# Give the configuration and weight files for the model and load the network.
net = cv2.dnn.readNet(path + 'yolov3.weights',
                path + 'yolov3.cfg')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)

In [ ]:
def searching(image, output_layers):
    admissible_crossing = 0.5
    confidence_level = 0.01
    H, W = image.shape[:2]

    boxes = []
    confidences = []
    probabilities = []

    for layer in output_layers:
        probability = layer[5:]
        index_max_probability = np.argmax(probability)
        confidence = probability[index_max_probability]
        if confidence > confidence_level:
            x, y, w, h = layer[:4] * np.array([W, H, W, H])
            p0 = int(x - w//2), int(y - h//2)
            p1 = int(x + w//2), int(y + h//2)
            boxes.append([*p0, int(w), int(h)])
            confidences.append(float(confidence))
            probabilities.append(index_max_probability)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_level, admissible_crossing)

    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[probabilities[i]]]
            image_new = cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(labels[probabilities[i]], confidences[i])
            image_new = cv2.putText(image_new, text, (x, y - 5), cv2.FONT_HERSHEY_TRIPLEX, 0.5, color, 1)
            return image_new

In [ ]:
def apply_yolo(image):
  height, width, depth = image.shape

  blob = cv2.dnn.blobFromImage(image, 1/255.0, (608, 608), swapRB = True, crop=False)
  net.setInput(blob)

  # determine the output layer
  layers_names = net.getLayerNames()
  unconnected_layers = net.getUnconnectedOutLayersNames()
  output_layers = np.vstack(net.forward(unconnected_layers))
  frame_new = searching(image, output_layers)
  return frame_new

In [ ]:
# Update video
size = (1280, 720)
codec = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter(path + 'video_new.avi', codec, 5, size)

cap = cv2.VideoCapture(video)

while cap.isOpened():
  print('.', end = '')

  ret, frame = cap.read()
  if not ret:
    break

  frame = apply_yolo(frame)
  writer.write(frame)

cap.release()
writer.release()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
# # Print in project
# cap = cv2.VideoCapture(path)

# while cap.isOpened():
#   ret, frame = cap.read()
#   if not ret:
#     break
#   pyplot.figure(figsize=(15, 10))

#   frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#   frame = apply_yolo(frame)

#   pyplot.imshow(frame)
#   display.clear_output(wait=True)
#   display.display(pyplot.gcf())

# cap.release()